In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from scipy import sparse 
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
import numbers
from multiprocessing import Pool
warnings.filterwarnings(action='once')

In [2]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [3]:
cnmfe_file_key = pd.read_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [4]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [5]:
tracking_files = ['/volumes/My_Passport/dlc_analysis/behavcamvideos/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [6]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in wt_CNMFE_file]
#CNMFE_dir_paths_list

In [7]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## spatial clustering

In [8]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [9]:
# compile and filter fluorescence
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))


In [ ]:
# clustering info per session 
coactivity_dfs = {}
for session in tqdm(list(grouped_raw_data.keys())):
    # get indicies of small cells 
    cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
    #compile and filter cell centers of mass
    filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

    com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score threshold
    cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
    binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

    reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

    #ks_results_2sided, ks_one_sided_more, ks_one_sided_less, coactivity_dfs = utils_jjm.spatial_coordination_by_session(reindexed, 10, com_distances)
    coactivity_dfs[session] = utils_jjm.spatial_coordination_by_session(reindexed)
    
    



  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [12:32<2:30:31, 752.62s/it]

 15%|█▌        | 2/13 [33:32<2:45:51, 904.71s/it]

 23%|██▎       | 3/13 [1:28:07<4:29:20, 1616.03s/it]

 31%|███       | 4/13 [1:37:02<3:13:43, 1291.52s/it]

 38%|███▊      | 5/13 [3:24:22<6:18:09, 2836.24s/it]

 46%|████▌     | 6/13 [5:56:35<9:11:16, 4725.17s/it]

 54%|█████▍    | 7/13 [8:13:33<9:37:17, 5772.97s/it]

 62%|██████▏   | 8/13 [8:38:31<6:14:13, 4490.64s/it]

 69%|██████▉   | 9/13 [8:53:02<3:46:58, 3404.72s/it]

 77%|███████▋  | 10/13 [9:08:38<2:13:11, 2663.98s/it]

 85%|████████▍ | 11/13 [10:52:23<2:04:24, 3732.25s/it]

 92%|█████████▏| 12/13 [12:06:16<1:05:42, 3942.52s/it]

In [24]:
len(reindexed)

6250

In [30]:
coactivity_dfs['GRIN013_H13_M33_S54'][1]

IndexError: list index out of range

In [10]:
session='GRIN013_H13_M33_S54'

cells_to_drop = np.array([cell for cell in range(1, len(grouped_raw_data[session]['cell_contours'])+1) if 
                              len(np.array(np.where(grouped_raw_data[session]['for_dims'][cell]>0.6)[0]))<100])
filtered_centers_of_mass = grouped_raw_data[session]['com'].drop(cells_to_drop, axis =0)

com_distances = utils_jjm.get_pairwise_distance_by_session(filtered_centers_of_mass)

    ##get binned fluorescence and calc Jaccard scores
    #arguments are sample widths to bin and z score thresholds
cells_in_session = C_df.loc[session].dropna(axis=1).drop('msCamFrame', axis=1)
binned_fluorescence = cells_in_session.apply(utils_jjm.binning_function_uncrop, args=[1, 2])

In [11]:
reindexed = binned_fluorescence.set_index(int(x) for x in np.linspace(0, len(binned_fluorescence)-1, len(binned_fluorescence)))

In [12]:
reindexed

,1,2,3,4,6,7,8,9,10,11,...,28,29,30,31,32,33,34,35,36,37
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6246,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [20]:
len(reindexed.columns)

36

In [14]:
coactivity_sparse = sparse.dok_matrix((36, 36), dtype=np.float32)

In [17]:
coactivity_sparse[6, 7] = 1

In [18]:
coactivity_sparse[6, 7]

1.0

In [32]:
cell_pairs = [pair for pair in itertools.combinations(list(reindexed.columns), 2)]

In [36]:
#cell_keys_matrix = sparse.dok_matrix((1, len(reindexed.columns)*len(reindexed.columns)), dtype=np.float32)
cell_keys_matrix = np.array(cell_pairs)

In [51]:
np.shape(cell_keys_matrix)

(630, 2)

In [44]:
#np.reshape(cell_keys_matrix, (36, 36))

In [46]:
d1_cellids = [pair[0] for pair in cell_pairs]
d2_cellids = [pair[1] for pair in cell_pairs]

In [48]:
len(d1_cellids)

630

In [ ]:
len(d2_cellids)

In [83]:
#parallel implementation
def map_to_sparse_matrix(item_idx_tuple, current_row_num, sparse_matrix, orig_df_comparison):
    if orig_df_comparison.loc[current_row_num][item[0]] == 1 and (orig_df_comparison.loc[current_row_num][item[0]] == 1):
        sparse_matrix[current_row_num, item_idx] = 1
    else:
        pass
    return(sparse_matrix)

def map_to_sparse_matrix_time(time_index):
    #for time_index in tqdm(range(len(reindexed))):
    #coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)
    for pair, pair_idx in tqdm(zip(cell_pairs, range(len(cell_pairs)))):
        if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
            coactivity_in_session[time_index, pair_idx] = 1
        else:
            pass
    return(coactivity_in_session)
    

In [ ]:
cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
coactivity_in_session_p = sparse.dok_matrix((len(reindexed), len(cell_pairs)))

p=Pool(3)

coactivity_in_session = p.map(map_to_sparse_matrix_time, range(len(reindexed)))




    
    

630it [00:00, 2736.80it/s]
630it [00:00, 2634.36it/s]
630it [00:00, 2520.55it/s]
630it [00:00, 3483.45it/s]
630it [00:00, 3339.06it/s]
630it [00:00, 2608.09it/s]
630it [00:00, 2783.90it/s]
630it [00:00, 2538.44it/s]
630it [00:00, 2548.27it/s]
630it [00:00, 3324.56it/s]
630it [00:00, 2615.66it/s]
630it [00:00, 2728.49it/s]
630it [00:00, 2923.42it/s]
630it [00:00, 2861.65it/s]
630it [00:00, 2721.66it/s]
630it [00:00, 3025.96it/s]
630it [00:00, 2963.13it/s]
630it [00:00, 2579.43it/s]
630it [00:00, 3736.81it/s]
630it [00:00, 2533.14it/s]
630it [00:00, 2633.52it/s]
630it [00:00, 2934.15it/s]
630it [00:00, 3124.97it/s]
630it [00:00, 3449.71it/s]
630it [00:00, 3104.87it/s]
630it [00:00, 2836.16it/s]
630it [00:00, 2612.81it/s]
630it [00:00, 2826.13it/s]
630it [00:00, 2228.53it/s]
630it [00:00, 2756.98it/s]
630it [00:00, 3086.01it/s]
630it [00:00, 3214.21it/s]
630it [00:00, 2820.66it/s]
630it [00:00, 2630.19it/s]
630it [00:00, 2848.60it/s]
630it [00:00, 2570.63it/s]
630it [00:00, 2808.04it/s]
6

630it [00:00, 3224.77it/s]
630it [00:00, 3631.86it/s]
630it [00:00, 3631.00it/s]
630it [00:00, 3906.06it/s]
630it [00:00, 3877.23it/s]
630it [00:00, 3764.82it/s]
630it [00:00, 4053.37it/s]
630it [00:00, 3568.71it/s]
630it [00:00, 3372.00it/s]
630it [00:00, 3706.67it/s]
630it [00:00, 3517.65it/s]
630it [00:00, 4001.17it/s]
630it [00:00, 3843.08it/s]
630it [00:00, 3927.97it/s]
630it [00:00, 3540.84it/s]
630it [00:00, 3773.75it/s]
630it [00:00, 3518.51it/s]
630it [00:00, 4073.80it/s]
630it [00:00, 3582.09it/s]
630it [00:00, 3759.82it/s]
630it [00:00, 4135.84it/s]
630it [00:00, 4140.21it/s]
630it [00:00, 3479.42it/s]
630it [00:00, 3639.65it/s]
630it [00:00, 3782.74it/s]
630it [00:00, 3943.34it/s]
630it [00:00, 3973.66it/s]
630it [00:00, 3520.69it/s]
630it [00:00, 3591.71it/s]
630it [00:00, 3868.00it/s]
630it [00:00, 3680.05it/s]
630it [00:00, 4414.19it/s]
630it [00:00, 4065.51it/s]
630it [00:00, 3825.19it/s]
630it [00:00, 3643.53it/s]
630it [00:00, 3760.48it/s]
630it [00:00, 3908.65it/s]
6

630it [00:00, 3511.94it/s]
630it [00:00, 3568.33it/s]
630it [00:00, 3787.51it/s]
630it [00:00, 3693.89it/s]
630it [00:00, 3720.18it/s]
630it [00:00, 3679.00it/s]
630it [00:00, 3572.00it/s]
630it [00:00, 3645.77it/s]
630it [00:00, 3217.17it/s]
630it [00:00, 3477.29it/s]
630it [00:00, 3404.89it/s]
630it [00:00, 3699.25it/s]
630it [00:00, 4211.97it/s]
630it [00:00, 4638.56it/s]
630it [00:00, 4036.29it/s]
630it [00:00, 3566.48it/s]
630it [00:00, 3913.41it/s]
630it [00:00, 2794.97it/s]
630it [00:00, 3615.56it/s]
630it [00:00, 3680.79it/s]
630it [00:00, 3517.00it/s]
630it [00:00, 3914.41it/s]
630it [00:00, 3505.96it/s]
630it [00:00, 3461.54it/s]
630it [00:00, 3460.61it/s]
630it [00:00, 3507.09it/s]
630it [00:00, 3061.38it/s]
630it [00:00, 3827.25it/s]
630it [00:00, 3563.33it/s]
630it [00:00, 3766.97it/s]
630it [00:00, 3506.51it/s]
630it [00:00, 3786.24it/s]
630it [00:00, 3629.05it/s]
630it [00:00, 4242.25it/s]
630it [00:00, 4022.52it/s]
630it [00:00, 3808.05it/s]
630it [00:00, 3829.23it/s]
6

630it [00:00, 4809.48it/s]
630it [00:00, 3980.27it/s]
630it [00:00, 3502.04it/s]
630it [00:00, 3644.38it/s]
630it [00:00, 4237.57it/s]
630it [00:00, 3511.66it/s]
630it [00:00, 3550.77it/s]
630it [00:00, 3466.28it/s]
630it [00:00, 3619.86it/s]
630it [00:00, 3596.36it/s]
630it [00:00, 3664.97it/s]
630it [00:00, 4046.76it/s]
630it [00:00, 3939.69it/s]
630it [00:00, 3857.41it/s]
630it [00:00, 3687.15it/s]
630it [00:00, 3317.15it/s]
630it [00:00, 3486.58it/s]
630it [00:00, 3839.68it/s]
630it [00:00, 3934.70it/s]
630it [00:00, 3705.27it/s]
630it [00:00, 4087.43it/s]
630it [00:00, 3741.05it/s]
630it [00:00, 3771.94it/s]
630it [00:00, 3975.54it/s]
630it [00:00, 3710.69it/s]
630it [00:00, 3571.03it/s]
630it [00:00, 3673.40it/s]
630it [00:00, 3560.45it/s]
630it [00:00, 3764.92it/s]
630it [00:00, 3779.99it/s]
630it [00:00, 3596.65it/s]
630it [00:00, 3534.46it/s]
630it [00:00, 3306.89it/s]
630it [00:00, 3571.57it/s]
630it [00:00, 3487.96it/s]
630it [00:00, 3749.13it/s]
630it [00:00, 4106.38it/s]
6

In [75]:
cell_pairs = np.array([pair for pair in itertools.combinations(list(reindexed.columns), 2)])
coactivity_in_session = sparse.dok_matrix((len(reindexed), len(cell_pairs)))
for time_index in tqdm(range(len(reindexed))):
    #coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)
    for pair, pair_idx in zip(cell_pairs, range(len(cell_pairs))):
        if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
            coactivity_in_session[time_index, pair_idx] = 1
        else:
            pass 
    



100%|██████████| 6250/6250 [09:12<00:00, 11.30it/s]


In [57]:
coactivity_by_time_point = sparse.dok_matrix((1, len(cell_pairs)), dtype=np.float32)

In [73]:
coactivity_in_session[69, 4]

1.0

In [65]:
outs = utils_jjm.spatial_coordination_by_session(reindexed)

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/abc.py", line 184, in __instancecheck__
    if subclass in cls._abc_cache:
  File "/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/_weakrefset.py", line 70, in __contains__
    def __contains__(self, item):
KeyboardInterrupt


KeyboardInterrupt: 

In [42]:
outs

[           1                              ... 33          34       35    36
            2  3  4  6  7  8  9  10 11 12  ... 34 35 36 37 35 36 37 36 37 37
 coactivity  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0
 
 [1 rows x 630 columns]]

In [50]:
reindexed

,1,2,3,4,6,7,8,9,10,11,...,28,29,30,31,32,33,34,35,36,37
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6246,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
#make following code work with these variables for function
binnums = 10

In [ ]:
ks_results_2sided = []
ks_one_sided_more = []
ks_one_sided_less = []

time_points = tqdm(range(len(reindexed)))
for time_index in time_points:
    coactivity_by_time_point = {}
    for pair in itertools.combinations(list(binned_fluorescence.columns), 2):
        if (reindexed.loc[time_index][pair[0]] == 1) and (reindexed.loc[time_index][pair[1]] == 1):
            coactivity_by_time_point[pair] = 1
        else:
            coactivity_by_time_point[pair] = 0
    coactivity_df = pd.DataFrame(coactivity_by_time_point, index=['coactivity'])

    coactive_cell_distances = pairwise_distance[coactivity_df[coactivity_df == 1].dropna(axis=1).columns]
    non_coactive_distances = pairwise_distance[coactivity_df[coactivity_df == 0].dropna(axis=1).columns]

    #linear_distribution = np.linspace(1, len(coactive_cell_distances.columns), binnums)

    #binnums=10
    cum_results_coactive = stats.cumfreq(coactive_cell_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))
    cum_results_non_coactive = stats.cumfreq(non_coactive_distances.values[0], numbins=binnums, defaultreallimits=(0, 500))

    #plt.plot(np.linspace(0, 500, binnums), cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #plt.plot(np.linspace(0, 500, binnums), linear_distribution/len(coactive_cell_distances.values[0]))
    #plt.show()

    less_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), 'norm', alternative='less')
    more_result = stats.kstest(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), 'norm', alternative='more')
    ks_one_sided_more.append(more_result)
    ks_one_sided_less.append(less_result)
    
    #coordination_index = utils_jjm.create_coordination_index(more_result, less_result)

    #two sided test between coactive and noncoactive distribution 
    ks_result = stats.ks_2samp(cum_results_coactive.cumcount/len(coactive_cell_distances.values[0]), cum_results_non_coactive.cumcount/len(non_coactive_distances.values[0]))
    #coordination_indicies.append(coordination_index)
    ks_results_2sided.append(ks_result)
    

In [ ]:
## for plotting average spatial coordination during trigger regions

In [ ]:

triggered_activity_across_sessions = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 40)
    
    f_threshold_activity = utils_jjm.average_triggered_regions(spatial_coordination_activity_in_session 
            , crossing_indicies, 40)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'spatial_coordination'])

In [ ]:
#triggered_activity_across_sessions

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

In [ ]:
combined_by_mouse.head()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean=combined_by_mouse.mean(axis=1, level=2)['velocity'].values
plt.plot(x, combined_by_mouse.mean(axis=1, level=2)['velocity'].values, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['velocity'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=(0), linestyle='--', color='r')
plt.show()

In [ ]:
x = np.linspace(-40*.2, 40*.2, 80)
mean = combined_by_mouse.mean(axis=1, level=2)['fluorescence'].values
plt.plot(x, mean, color='k')
std_error = (combined_by_mouse.std(axis=1, level=2)['fluorescence'])/math.sqrt(combined_by_mouse.mean(axis=1, level=0).shape[1])
plt.fill_between(x, mean-std_error, mean+std_error)
ax = plt.gca()
ax.axvline(x=0, linestyle='--', color='r')
plt.show()